In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from IPython.display import display

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression

In [2]:
import scipy.sparse
import pickle
import xgboost as xgb
from xgboost import plot_importance
from matplotlib import pyplot
from sklearn.feature_extraction.text import CountVectorizer

In [3]:
from xgboost.sklearn import XGBClassifier
from sklearn import cross_validation, metrics   #Additional scklearn functions
from sklearn.grid_search import GridSearchCV   #Perforing grid search

import matplotlib.pylab as plt
%matplotlib inline


/Users/bonnieliu/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/bonnieliu/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [4]:
row_train = pd.read_csv("/Users/bonnieliu/Downloads/OneDrive_2018-02-26/train.csv", sep=',')
row_val = pd.read_csv("/Users/bonnieliu/Downloads/OneDrive_2018-02-26/validation.csv", sep=',')
row_test = pd.read_csv("/Users/bonnieliu/Downloads/OneDrive_2018-02-26/test.csv", sep=',')

In [5]:
row_train.useragent.unique()

array(['windows_ie', 'windows_chrome', 'mac_other', 'windows_other',
       'android_safari', 'mac_safari', 'windows_maxthon',
       'windows_firefox', 'mac_chrome', 'linux_firefox', 'windows_opera',
       'windows_theworld', 'android_other', 'windows_safari', 'ios_safari',
       'linux_other', 'mac_firefox', 'android_opera', 'mac_maxthon',
       'other_other', 'android_chrome', 'linux_chrome', 'other_opera',
       'linux_safari', 'other_ie', 'ios_other', 'linux_opera',
       'android_sogou', 'other_safari', 'windows_sogou', 'mac_opera',
       'mac_sogou', 'android_maxthon', 'android_firefox', 'linux_ie',
       'other_chrome', 'android_ie', 'other_firefox'], dtype=object)

# Feature engineering

In [6]:
#train set usetag
data_usertag=row_train.usertag.fillna('0')
data_usertag=data_usertag.str.replace(',',' ')
vect=CountVectorizer()
data_usertag_vect=vect.fit_transform(data_usertag)
usertag_train=pd.DataFrame(data_usertag_vect.toarray(),columns=vect.get_feature_names())

In [7]:
#val set usertag
data_usertag=row_val.usertag.fillna('0')
data_usertag=data_usertag.str.replace(',',' ')
vect=CountVectorizer()
data_usertag_vect=vect.fit_transform(data_usertag)
usertag_val=pd.DataFrame(data_usertag_vect.toarray(),columns=vect.get_feature_names())

In [8]:
#test set usertag
data_usertag=row_test.usertag.fillna('0')
data_usertag=data_usertag.str.replace(',',' ')
vect=CountVectorizer()
data_usertag_vect=vect.fit_transform(data_usertag)
usertag_test=pd.DataFrame(data_usertag_vect.toarray(),columns=vect.get_feature_names())

In [9]:
hour_mapping = {  
           1: 'midnight',2: 'midnight',3: 'midnight',4: 'midnight',5: 'midnight',6: 'midnight',7: 'midnight',8: 'midnight',  
           9: 'day',10: 'day',11: 'day',12: 'day',13: 'day',14: 'day',15: 'day',16: 'day' , 
           17: 'night',18: 'night',19: 'night',20: 'night',21: 'night',22: 'night',23: 'night',0: 'night'}  
row_train['hour'] = row_train['hour'].map(hour_mapping)  

In [10]:
row_val['hour'] = row_val['hour'].map(hour_mapping)  
row_test['hour'] = row_test['hour'].map(hour_mapping)  

In [11]:
row_train.columns

Index(['click', 'weekday', 'hour', 'bidid', 'userid', 'useragent', 'IP',
       'region', 'city', 'adexchange', 'domain', 'url', 'urlid', 'slotid',
       'slotwidth', 'slotheight', 'slotvisibility', 'slotformat', 'slotprice',
       'creative', 'bidprice', 'payprice', 'keypage', 'advertiser', 'usertag'],
      dtype='object')

In [12]:
useragent_mapping = {  
    'windows_ie':'ie', 
    'windows_chrome':'chrome', 
    'mac_other':'other', 
    'windows_other':'other',
    'android_safari':'safari',
    'mac_safari':'safari', 
    'windows_maxthon':'maxthon',
    'windows_firefox':'firefox', 
    'mac_chrome':'chrome', 
    'linux_firefox':'firefox', 
    'windows_opera':'opera',
    'windows_theworld':'theworld', 
    'android_other':'other', 
    'windows_safari':'safari',
    'ios_safari':'safari',
    'linux_other':'other',
    'mac_firefox':'firefox', 
    'android_opera':'opera', 
    'mac_maxthon':'maxthon',
    'other_other':'other', 
    'android_chrome':'chrome',
    'linux_chrome':'chrome',
    'other_opera':'opera',
    'linux_safari':'safari',
    'other_ie':'ie', 
    'ios_other':'other',
    'linux_opera':'opera',
    'android_sogou':'sogou', 
    'other_safari':'safari', 
    'windows_sogou':'sogou', 
    'mac_opera':'opera',
    'mac_sogou':'sogou', 
    'android_maxthon':'maxthon',
    'android_firefox':'firefox',
    'linux_ie':'ie',
    'other_chrome':'chrome',
    'android_ie':'ie', 
    'other_firefox':'firefox'}  
row_train['browse'] = row_train['useragent'].map(useragent_mapping)  

In [13]:
row_val['browse'] = row_val['useragent'].map(useragent_mapping)  
row_test['browse'] = row_test['useragent'].map(useragent_mapping)  

In [14]:
useragent_mapping1 = {  
    'windows_ie':'windows', 
    'windows_chrome':'windows', 
    'mac_other':'mac', 
    'windows_other':'windows',
    'android_safari':'android',
    'mac_safari':'mac', 
    'windows_maxthon':'windows',
    'windows_firefox':'windows', 
    'mac_chrome':'mac', 
    'linux_firefox':'linux', 
    'windows_opera':'windows',
    'windows_theworld':'windows', 
    'android_other':'android', 
    'windows_safari':'windows',
    'ios_safari':'ios',
    'linux_other':'linux',
    'mac_firefox':'mac', 
    'android_opera':'android', 
    'mac_maxthon':'mac',
    'other_other':'other', 
    'android_chrome':'android',
    'linux_chrome':'linux',
    'other_opera':'other',
    'linux_safari':'linux',
    'other_ie':'other', 
    'ios_other':'ios',
    'linux_opera':'linux',
    'android_sogou':'android', 
    'other_safari':'other', 
    'windows_sogou':'windows', 
    'mac_opera':'mac',
    'mac_sogou':'mac', 
    'android_maxthon':'android',
    'android_firefox':'android',
    'linux_ie':'linux',
    'other_chrome':'other',
    'android_ie':'android', 
    'other_firefox':'other'}  
row_train['agent'] = row_train['useragent'].map(useragent_mapping1)  

In [15]:
row_val['agent'] = row_val['useragent'].map(useragent_mapping1)  
row_test['agent'] = row_test['useragent'].map(useragent_mapping1)  

In [16]:
slotvisibility_mapping = {  
    '0':0, 
    '1':1, 
    '2':2, 
    '255':4,
    'FifthView':5,
    'FirstView':6, 
    'FourthView':7,
    'Na':8, 
    'OtherView':9, 
    'SecondView':10, 
    'ThirdView':11,
    }  
row_train['slotview'] = row_train['slotvisibility'].map(slotvisibility_mapping)  

In [17]:
row_val['slotview'] = row_val['slotvisibility'].map(slotvisibility_mapping)
row_test['slotview'] = row_test['slotvisibility'].map(slotvisibility_mapping)  

In [18]:
#check missing
row_train.apply(lambda x: sum(x.isnull()))

click             0
weekday           0
hour              0
bidid             0
userid            0
useragent         0
IP                0
region            0
city              0
adexchange        0
domain            0
url               0
urlid             0
slotid            0
slotwidth         0
slotheight        0
slotvisibility    0
slotformat        0
slotprice         0
creative          0
bidprice          0
payprice          0
keypage           0
advertiser        0
usertag           0
browse            0
agent             0
slotview          0
dtype: int64

In [19]:
row_train.head()

,click,weekday,hour,bidid,userid,useragent,IP,region,city,adexchange,...,slotprice,creative,bidprice,payprice,keypage,advertiser,usertag,browse,agent,slotview
0,0,5,night,b7bea80521fdecd95d2d761a38c91c3f09618066,2e880fb7d690cf7377b2e42e701728e3f3c0e4c1,windows_ie,125.37.175.*,2,2,2,...,5,a4f763f78ef3eedfe614263b94a8924e,238,5,0f951a030abdaedd733ee8d114ce2944,3427,null,ie,windows,2
1,0,1,night,4f51205475678f5a124bc76b2c54163bf8eaa7eb,3a1fe01360ff8100e7d006b83b77a3e4c01d928c,windows_chrome,171.36.92.*,238,239,1,...,0,10722,294,23,null,2821,null,chrome,windows,7
2,0,3,day,b604e3fd054a658ab7ced4285ebf2ef54d2bd890,801d18a056b6fe6b06a794aef17fb0d6daff2414,windows_ie,59.46.106.*,40,41,2,...,5,798b2d49952d77f1eace9f23c210d0b5,238,24,0f951a030abdaedd733ee8d114ce2944,3427,"10052,10006,13866,10110",ie,windows,2
3,0,6,night,0348beeae93e561584c3b50fc9e7746a33048ad7,0d6eaf2259699990e38a1fc5116f112070b9ecdc,windows_ie,114.250.226.*,1,1,1,...,0,cb7c76e7784031272e37af8e7e9b062c,300,25,bebefa5efe83beee17a3d245e7c5085b,1458,"13866,10063,10111",ie,windows,2
4,0,5,midnight,268149c1789bce2bc9798ffd97ec431219bafeb3,a239d9bb642460d974ba67f85e63b8d3e214da0e,windows_ie,183.63.192.*,216,233,2,...,133,7330,277,133,null,2259,null,ie,windows,9


In [20]:
row_train.dtypes

click              int64
weekday            int64
hour              object
bidid             object
userid            object
useragent         object
IP                object
region             int64
city               int64
adexchange        object
domain            object
url               object
urlid             object
slotid            object
slotwidth          int64
slotheight         int64
slotvisibility    object
slotformat        object
slotprice          int64
creative          object
bidprice           int64
payprice           int64
keypage           object
advertiser         int64
usertag           object
browse            object
agent             object
slotview           int64
dtype: object

In [21]:
def onehot_encoding(data,cols):
    for col in cols:
        df_temp = pd.get_dummies(data[col],prefix=col, drop_first=True)
        data = data.join(df_temp)
        
    return data

In [22]:
not_contains = ['click','bidid', 'userid', 'useragent', 'IP',
        'city', 'adexchange', 'domain', 'url', 'urlid', 'slotid',
        'slotformat', 'slotprice','slotview','weekday','advertiser','region','city','slotwidth',
       'creative', 'bidprice', 'payprice', 'keypage',  'usertag','hour','browse','agent','slotvisibility']

In [23]:
columns=['hour', 'slotview','weekday','advertiser',
       'browse','agent','region','slotwidth']

In [24]:
x_train1 = onehot_encoding(row_train,columns)

In [25]:
y_train = row_train['click']
x_val1 = onehot_encoding(row_val,columns)
y_val = row_val['click']
x_test1 = onehot_encoding(row_test,columns)

In [26]:
def get_x_features(data):
    columns_train = data.columns
    columns_train_values = np.array(columns_train)
    train_cols = []
    for col in columns_train_values:
        if col not in not_contains:
            train_cols.append(col)
    return train_cols

In [27]:
x_train = get_x_features(x_train1)
trainData = x_train1.loc[:,x_train]
x_train = trainData
x_train = pd.concat([x_train,usertag_train], axis=1)
x_train.head()

,slotheight,hour_midnight,hour_night,slotview_1,slotview_2,slotview_4,slotview_5,slotview_6,slotview_7,slotview_8,...,13874,14273,15398,16593,16617,16661,16706,16751,16753,null
0,200,0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,250,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
2,250,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,600,0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,90,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [28]:
x_val1 = onehot_encoding(row_val,columns)
x_test1 = onehot_encoding(row_test,columns)

In [29]:
x_val = get_x_features(x_val1)
x_test = get_x_features(x_test1)
x_val = x_val1.loc[:,x_val]
x_test = x_test1.loc[:,x_test]

In [30]:
x_val = pd.concat([x_val,usertag_val], axis=1)
x_test = pd.concat([x_test,usertag_test], axis=1)

In [31]:
x_val.head()

,slotheight,hour_midnight,hour_night,slotview_1,slotview_2,slotview_4,slotview_5,slotview_6,slotview_7,slotview_8,...,13874,14273,15398,16593,16617,16661,16706,16751,16753,null
0,600,0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,90,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,250,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,280,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,200,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Sample

In [32]:
data = pd.concat([y_train,x_train], axis=1)

In [33]:
df = data
df1 = df.groupby(['click']).get_group(0)
df2 = df.groupby(['click']).get_group(1)
click0 = len(df1)
click1 = len(df2)
print(click0)
click1

2429188


1793

In [34]:
#frac_rate = num(0:1) 
def sample (frac_rate):
    numclick0 = click1*frac_rate 
    dfclick0 = df1.sample(n = numclick0)
    dfclick1 = df2.sample(n = 1793)
    frames = [dfclick0,dfclick1]
    result = pd.concat(frames)
    return result

In [35]:
sample1 = 

sample(50)
len(sample1)

91443

In [36]:
sample_y = pd.DataFrame(sample1,columns=['click'])
sample_x = sample1.drop(['click'],axis=1)
sample_x.head()

,slotheight,hour_midnight,hour_night,slotview_1,slotview_2,slotview_4,slotview_5,slotview_6,slotview_7,slotview_8,...,13874,14273,15398,16593,16617,16661,16706,16751,16753,null
224827,280,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2238536,90,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1395970,250,0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1330632,280,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
397740,250,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [38]:
sample_x.columns

Index(['slotheight', 'hour_midnight', 'hour_night', 'slotview_1', 'slotview_2',
       'slotview_4', 'slotview_5', 'slotview_6', 'slotview_7', 'slotview_8',
       ...
       '13874', '14273', '15398', '16593', '16617', '16661', '16706', '16751',
       '16753', 'null'],
      dtype='object', length=163)

# modelling

In [37]:
from sklearn.externals import joblib

In [43]:
x_train = sample_x.reset_index(drop=True)
y_train = sample_y.reset_index(drop=True)
x_train.head()

,slotheight,hour_midnight,hour_night,slotview_1,slotview_2,slotview_4,slotview_5,slotview_6,slotview_7,slotview_8,...,13874,14273,15398,16593,16617,16661,16706,16751,16753,null
0,280,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,90,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,250,0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,280,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,250,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [77]:
#Set parameter
xgb1 = XGBClassifier(
 learning_rate =0.1,
 n_estimators=118,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=27,
 slient=True)


In [78]:
#Model
xgb_param = xgb1.get_xgb_params()
xgtrain = xgb.DMatrix(x_train, label=y_train)
cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=xgb1.get_params()['n_estimators'], nfold=5,
            metrics='auc',early_stopping_rounds=12,)
xgb1.set_params(n_estimators=cvresult.shape[0])
print('running cross validation, disable standard deviation display')
print(cvresult.shape[0])
        
#Fit the algorithm on the data
xgb1.fit(x_train, y_train.values.ravel(),eval_metric='auc')
        
#Predict validation set:
dval_predictions = xgb1.predict(x_val)
dval_predprob = xgb1.predict_proba(x_val)[:,1]

#Predict test set:
dtest_predictions = xgb1.predict(x_test)
dtest_predprob = xgb1.predict_proba(x_test)[:,1]
    
#Save model
joblib.dump(xgb1, 'xgb1.model')
#Print model report:
print ("\nModel Report")
print("Accuracy : %.4g" % metrics.accuracy_score(y_val, dval_predictions))
print ("AUC Score (Validation): %f" % metrics.roc_auc_score(y_val, dval_predprob))
                    


running cross validation, disable standard deviation display
118

Model Report
Accuracy : 0.9987
AUC Score (Validation): 0.872010


In [75]:
#Save model
joblib.dump(xgb1, 'xgb_169.model')
#Load model
xgb = joblib.load('xgb_169.model')

['xgb_169.model']

In [114]:
len(dval_predprob)

303925

In [65]:
#Prepare validation
dval_predprob = pd.DataFrame(dval_predprob,columns=['pCTR'])
predCTR=dval_predprob

# Step2 Tune max_depth & min_child_weight

In [138]:
param_test1 = {
 'max_depth':[3,5,7,9],
 'min_child_weight':[1,3,5]
}
 

In [140]:
#Remember : use optimal n_estimators calculated above
gsearch1 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=5,
 min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27), 
 param_grid = param_test1, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch1.fit(x_train,y_train)
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

([mean: 0.70730, std: 0.00588, params: {'max_depth': 3, 'min_child_weight': 1},
  mean: 0.70692, std: 0.00683, params: {'max_depth': 3, 'min_child_weight': 3},
  mean: 0.70730, std: 0.00718, params: {'max_depth': 3, 'min_child_weight': 5},
  mean: 0.70543, std: 0.00892, params: {'max_depth': 5, 'min_child_weight': 1},
  mean: 0.70360, std: 0.00877, params: {'max_depth': 5, 'min_child_weight': 3},
  mean: 0.70390, std: 0.00761, params: {'max_depth': 5, 'min_child_weight': 5},
  mean: 0.69360, std: 0.01109, params: {'max_depth': 7, 'min_child_weight': 1},
  mean: 0.69138, std: 0.00963, params: {'max_depth': 7, 'min_child_weight': 3},
  mean: 0.69520, std: 0.00892, params: {'max_depth': 7, 'min_child_weight': 5},
  mean: 0.68109, std: 0.01032, params: {'max_depth': 9, 'min_child_weight': 1},
  mean: 0.67911, std: 0.01062, params: {'max_depth': 9, 'min_child_weight': 3},
  mean: 0.68718, std: 0.00763, params: {'max_depth': 9, 'min_child_weight': 5}],
 {'max_depth': 3, 'min_child_weight': 5

In [ ]:
#Narrow down
param_test2 = {
 'max_depth':[4,5,6],
 'min_child_weight':[4,5,6]
}
gsearch2 = GridSearchCV(estimator = XGBClassifier( learning_rate=0.1, n_estimators=140, max_depth=5,
 min_child_weight=2, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
 param_grid = param_test2, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch2.fit(train[predictors],train[target])
gsearch2.grid_scores_, gsearch2.best_params_, gsearch2.best_score_

# Step 3: Tune gamma

In [ ]:
#Remember : use optimal max_depth & min_child_weight calculated above
param_test3 = {
 'gamma':[i/10.0 for i in range(0,5)]
}
gsearch3 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=3,
 min_child_weight=5, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
 param_grid = param_test3, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch3.fit(train[predictors],train[target])
gsearch3.grid_scores_, gsearch3.best_params_, gsearch3.best_score_

# Step 4: Tune subsample and colsample_bytree

In [ ]:
#Remember : use optimal gamma calculated above
param_test4 = {
 'subsample':[i/10.0 for i in range(6,10)],
 'colsample_bytree':[i/10.0 for i in range(6,10)]
}
gsearch4 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=177, max_depth=4,
 min_child_weight=6, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
 param_grid = param_test4, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch4.fit(train[predictors],train[target])
gsearch4.grid_scores_, gsearch4.best_params_, gsearch4.best_score_

# Step 5: Reducing Learning Rate

In [ ]:
#Remember : use optimal msubsample and colsample_bytree calculated above

In [100]:
xgb2 = xgb.XGBClassifier(
 booster='gbtree',
            learning_rate = 0.01,
            n_estimators=69,
            max_depth=8,
            min_child_weight=6,
            gamma=0.1,
            subsample=0.6,
            colsample_bytree=0.85,
            objective= 'binary:logistic',
            nthread=4,
            scale_pos_weight=1, 
            seed=27,
            n_jobs=4)

In [ ]:
#from sklearn.externals import joblib
#xgb = model_XGB.fit(x_train, y_train)
#joblib.dump(xgb, 'xgb.model')
#xgb = joblib.load('xgb.model')

# Plot pCTR

In [ ]:

xgb_val = joblib.load('xgb1.model')
dval_predprob = xgb_val.predict_proba(x_val)
dval_predprob1 = xgb_val.predict_proba(x_val)[:,1]
dval_predprob0 = xgb_val.predict_proba(x_val)[:,0]


xgb_val11 = joblib.load('xgb11.model')
dval_predprob1_ = xgb_val11.predict_proba(x_val)
dval_predprob1_1 = xgb_val11.predict_proba(x_val)[:,1]
dval_predprob1_0 = xgb_val11.predict_proba(x_val)[:,0]

In [ ]:
# click=1 prediction as click = =1 probabilities
click_xgb1a=dval_predprob[dval_predprob1.astype(bool),1]
click_xgb1b=dval_predprob[dval_predprob0.astype(bool),0]

click1_xgb2a=dval_predprob1_[dval_predprob1_1.astype(bool),1]
click1_xgb2b=dval_predprob1_[dval_predprob1_0.astype(bool),0]


plt.figure(figsize = (10,7))


plt.subplot(221)

width = 0.6
pyplot.hist(click_xgb1a, bins = np.linspace(0.0, 0.06, 300), alpha=0.5, label='Model_xgb1')
plt.xlabel('probability', fontsize=10)
plt.ylabel('Frequency', fontsize=10)
plt.title('Predicted probabilities for clicks = 1', fontsize=13)
plt.axvline(0.0042, color = 'grey', linestyle = '--', linewidth = 1)
pyplot.legend(loc='upper center')


plt.subplot(222)
width = 0.6
pyplot.hist(click_xgb1b, bins = np.linspace(0.8, 1.00, 300), alpha=0.5, label='Model_xgb1')
plt.ylabel('Frequency', fontsize=10)
plt.title('Predicted probabilities for clicks = 0', fontsize=13)
plt.axvline(0.995, color = 'grey', linestyle = '--', linewidth = 1)
pyplot.legend(loc='upper center')

plt.subplot(223)
width = 0.6
pyplot.hist(click1_xgb2a, bins = np.linspace(0.0, 0.06, 300), alpha=0.5, label='Model_xgb2',color='green')
plt.ylabel('Frequency', fontsize=10)
plt.title('Predicted probabilities for clicks = 1', fontsize=13)
plt.axvline(0.0042, color = 'grey', linestyle = '--', linewidth = 1)
pyplot.legend(loc='upper center')

plt.subplot(224)
width = 0.6
pyplot.hist(click1_xgb2b, bins = np.linspace(0.8, 1.00, 300), alpha=0.5, label='Model_xgb2',color='green')
plt.ylabel('Frequency', fontsize=10)
plt.title('Predicted probabilities for clicks = 0', fontsize=13)
plt.axvline(0.996, color = 'grey', linestyle = '--', linewidth = 1)
pyplot.legend(loc='upper center')

plt.tight_layout()
plt.show()

# Validation part

In [52]:
#pre = pd.read_csv("/Users/bonnieliu/Downloads/pred_lr-2.csv", sep=',')
validation = pd.read_csv("/Users/bonnieliu/Downloads/OneDrive_2018-02-26/validation.csv", sep=',')
train = pd.read_csv("/Users/bonnieliu/Downloads/OneDrive_2018-02-26/train.csv", sep=',')
test = pd.read_csv("/Users/bonnieliu/Downloads/OneDrive_2018-02-26/test.csv", sep=',')


In [121]:
dval_predprob = pd.DataFrame(dval_predprob,columns=['pCTR'])
#dtest_predprob= pd.DataFrame(dtest_predprob,columns=['pCTR'])
predCTR=dval_predprob

In [55]:
#Training set aCTR
clicks = len(train[train.click==1])
impression = len(train[train.bidprice>train.payprice])
aCTR=clicks/impression
aCTR

0.0007383985358870749

In [128]:
def evaluate(bidprice):
    budget=6250000
    impression=0
    click=0
    for row in range(0,validation.shape[0]):
        if budget<validation.payprice[row]:
            continue
        if bidprice.iloc[row,0]>validation.payprice[row]:
            budget=budget-validation.payprice[row]
            impression=impression+1
            if validation.click[row]==1:
                click=click+1
    Cost = 6250000 - budget
    CTR = click/impression
    
    #CPM: Cost per mille 
    CPM = Cost/impression

    #eCPC: effective Cost per Click
    eCPC = Cost/clicks
    estimator = [click,impression,CTR,Cost,CPM,eCPC]
    return  estimator

In [129]:
def bid_linear(basebid, pred_CTR,avg_CTR):
    bid = np.zeros((validation.shape[0], 1))
    for i in range (0,validation.shape[0]):
        bid[i,0] = basebid*(predCTR.pCTR.iloc[i])/avg_CTR
    bid = pd.DataFrame(bid)
    return bid

In [136]:
#Search for best basebid
for counter in np.arange(4,8):
    base_bid=counter
    bidprice=bid_linear(base_bid, predCTR, aCTR)
    estimator = evaluate(bidprice)
    print('basebid:',base_bid, 'click:',estimator[0], 'impression:',estimator[1],
          'CTR:%.5f'% estimator[2], 'cost:%.1f'% estimator[3], 'CPM:%.2f'% estimator[4], 'eCPC:%.2f'% estimator[5])   

basebid: 4 click: 155 impression: 117490 CTR:0.00132 cost:4795623.0 CPM:40.82 eCPC:2674.64
basebid: 5 click: 160 impression: 139812 CTR:0.00114 cost:6249999.0 CPM:44.70 eCPC:3485.78
basebid: 6 click: 134 impression: 130253 CTR:0.00103 cost:6250000.0 CPM:47.98 eCPC:3485.78
basebid: 7 click: 119 impression: 123014 CTR:0.00097 cost:6250000.0 CPM:50.81 eCPC:3485.78


# Test

In [69]:
def bid_linear1(basebid, pred_CTR,avg_CTR):
    bid = np.zeros((test.shape[0], 1))
    for i in range (0,test.shape[0]):
        bid[i,0] = basebid*predCTR.pCTR.iloc[i]/avg_CTR
    bid = pd.DataFrame(bid)
    return bid

In [ ]:
dtest_predprob= pd.DataFrame(dtest_predprob,columns=['pCTR'])
predCTR=dtest_predprob

In [70]:
result = bid_linear1(5,predCTR, aCTR)
result.columns=['bidprice']
bidid = pd.DataFrame(test, columns=['bidid'])
final = pd.concat([bidid,result], axis=1)
final.head()

,bidid,bidprice
0,366c563de7d90feb9d4dab53e795a93fb3157387,140.316385
1,29167d4caa719788b5a342dbaa25151d53121f80,495.113554
2,ff8bc3f4d44a3ea60c5f3a3a8fbe7cd98fb2966e,26.502562
3,844c2da00d45315f20b748ec131c26ee99a7cbc7,102.889554
4,c6017f0ad0c44d7d0c9b62583ea863f28941c0ca,122.936968


In [43]:
final.groupby("bidprice").count()

,bidid
bidprice,
18.325756,64
18.697510,12
21.236616,114
22.045428,16
22.388442,2
22.458270,3
22.492621,5
23.241821,73
23.280079,54


In [71]:
final.to_csv("/Users/bonnieliu/Downloads/Group27xgb1.csv",index = False)

In [72]:
xg = pd.read_csv("/Users/bonnieliu/Downloads/Group27.csv",sep=',')

In [73]:
len(xg)

303375